In [ ]:
# Delete the cache file and re-run
import os
if os.path.exists('google_ip_ranges_cache.json'):
    os.remove('google_ip_ranges_cache.json')


In [1]:
import os
import json
import csv
import gzip
import socket
import time
import zipfile
import ipaddress
import requests
import dns.resolver
import dns.reversename
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from tqdm import tqdm
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================

MAX_DNS_WORKERS = 150
DNS_TIMEOUT = 2
CACHE_FILE = "verified_ips_cache.txt"
IP_RANGES_CACHE_FILE = "google_ip_ranges_cache.json"
CACHE_EXPIRY_HOURS = 24
GOOGLE_DOMAINS = ['.googlebot.com', '.google.com', '.googleusercontent.com']
MAX_RECORDS_PER_FILE = 500000

# NEW: Lenient mode and FcrDNS fallback
LENIENT_MODE = True  # Accept records with UA but missing/invalid IP
ENABLE_FCRDNS_FALLBACK = True  # Verify non-CIDR IPs with FcrDNS

GOOGLE_IP_RANGE_URLS = {
    'googlebot': 'https://developers.google.com/static/search/apis/ipranges/googlebot.json',
    'special_crawlers': 'https://developers.google.com/static/search/apis/ipranges/special-crawlers.json',
    'user_triggered_fetchers': 'https://developers.google.com/static/search/apis/ipranges/user-triggered-fetchers.json',
    'user_triggered_fetchers_google': 'https://developers.google.com/static/search/apis/ipranges/user-triggered-fetchers-google.json',
}

# ============================================================================
# FETCH DYNAMIC IP RANGES
# ============================================================================

def fetch_google_ip_ranges():
    """Fetch Google's official IP ranges (IPv4 + IPv6) from JSON files"""
    print("\n🌐 Fetching Google IP ranges from official sources...")
    
    all_prefixes = []
    
    for source_name, url in GOOGLE_IP_RANGE_URLS.items():
        try:
            print(f"   • Fetching {source_name}...")
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            data = response.json()
            
            if 'prefixes' in data:
                for prefix in data['prefixes']:
                    if 'ipv4Prefix' in prefix:
                        all_prefixes.append(prefix['ipv4Prefix'])
                    if 'ipv6Prefix' in prefix:
                        all_prefixes.append(prefix['ipv6Prefix'])
            
            print(f"     ✅ Retrieved {len(data.get('prefixes', []))} prefixes")
        
        except Exception as e:
            print(f"     ⚠️ Warning: Failed to fetch {source_name}: {str(e)}")
    
    ip_networks = []
    ipv4_count = 0
    ipv6_count = 0
    
    for prefix in all_prefixes:
        try:
            network = ipaddress.ip_network(prefix)
            ip_networks.append(network)
            
            if network.version == 4:
                ipv4_count += 1
            else:
                ipv6_count += 1
        except ValueError:
            continue
    
    print(f"\n✅ Total IP ranges loaded: {len(ip_networks)} CIDR blocks")
    print(f"   • IPv4 ranges: {ipv4_count}")
    print(f"   • IPv6 ranges: {ipv6_count}")
    return ip_networks

def load_or_fetch_ip_ranges(cache_file=IP_RANGES_CACHE_FILE):
    """Load IP ranges from cache or fetch from Google if expired"""
    if os.path.exists(cache_file):
        try:
            with open(cache_file, 'r') as f:
                cache_data = json.load(f)
            
            cache_time = cache_data.get('timestamp', 0)
            cache_age_hours = (time.time() - cache_time) / 3600
            
            if cache_age_hours < CACHE_EXPIRY_HOURS:
                print(f"\n✅ Using cached IP ranges (age: {cache_age_hours:.1f} hours)")
                ip_networks = [ipaddress.ip_network(cidr) for cidr in cache_data['prefixes']]
                
                ipv4_count = sum(1 for n in ip_networks if n.version == 4)
                ipv6_count = sum(1 for n in ip_networks if n.version == 6)
                print(f"   • IPv4 ranges: {ipv4_count}")
                print(f"   • IPv6 ranges: {ipv6_count}")
                return ip_networks
            else:
                print(f"\n⏰ Cache expired, fetching fresh data...")
        except Exception as e:
            print(f"\n⚠️ Cache error, fetching fresh data...")
    
    ip_networks = fetch_google_ip_ranges()
    
    try:
        cache_data = {
            'timestamp': time.time(),
            'prefixes': [str(network) for network in ip_networks]
        }
        with open(cache_file, 'w') as f:
            json.dump(cache_data, f, indent=2)
        print(f"💾 IP ranges cached")
    except Exception as e:
        print(f"⚠️ Failed to save cache")
    
    return ip_networks

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def extract_full_timestamp_from_filename(filename):
    """Extract unique timestamp from filename"""
    base_name = filename
    if base_name.endswith('.gz'):
        base_name = base_name[:-3]
    if base_name.endswith('.log'):
        base_name = base_name[:-4]
    if base_name.startswith('nginx-'):
        base_name = base_name[6:]
    return base_name if base_name else None

def extract_first_ip(ip_string):
    """Extract first IP (IPv4 or IPv6) from X-Forwarded-For field"""
    if not ip_string:
        return None
    
    ip = str(ip_string).split(',')[0].strip()
    
    if ip.startswith('[') and ip.endswith(']'):
        ip = ip[1:-1]
    
    return ip if ip else None

def is_googlebot(user_agent_str):
    """UA validation - Check if contains googlebot patterns"""
    if not user_agent_str:
        return False
    
    try:
        ua_lower = str(user_agent_str).lower()
    except:
        return False
    
    googlebot_patterns = [
        'googlebot', 'google-inspectiontool', 'googlebot-image', 'googlebot-news',
        'googlebot-video', 'adsbot-google', 'mediapartners-google', 'apis-google',
        'google favicon', 'feedfetcher-google', 'google-read-aloud', 'duplichecker',
        'google web preview', 'google-site-verification', 'google-smartphone'
    ]
    
    for pattern in googlebot_patterns:
        if pattern in ua_lower:
            return True
    return False

def create_zip_archive(file_list, output_folder, archive_name="googlebot_data"):
    """Create ZIP archive"""
    zip_filename = os.path.join(output_folder, f"{archive_name}_{time.strftime('%Y%m%d_%H%M%S')}.zip")
    
    print(f"\n📦 Creating ZIP archive...")
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in tqdm(file_list, desc="Archiving", unit=" files"):
            zipf.write(file_path, arcname=os.path.basename(file_path))
    
    zip_size_mb = os.path.getsize(zip_filename) / (1024**2)
    print(f"✅ ZIP created: {os.path.basename(zip_filename)} ({zip_size_mb:.2f} MB)")
    return zip_filename

# ============================================================================
# VERIFICATION METHODS
# ============================================================================

def verify_ip_in_range(ip, ip_networks):
    """Check if IP is in Google's CIDR ranges"""
    try:
        ip_addr = ipaddress.ip_address(ip)
        for network in ip_networks:
            if ip_addr in network:
                return True
        return False
    except ValueError:
        return False

def verify_ip_method2_dns_fast(ip):
    """FcrDNS Verification - ULTRA FAST using dnspython"""
    try:
        rev_name = dns.reversename.from_address(ip)
        
        resolver = dns.resolver.Resolver()
        resolver.timeout = DNS_TIMEOUT
        resolver.lifetime = DNS_TIMEOUT
        
        try:
            reverse_answers = resolver.resolve(rev_name, 'PTR')
            hostname = str(reverse_answers[0]).rstrip('.')
            hostname_lower = hostname.lower()
        except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer, dns.resolver.Timeout, dns.exception.DNSException):
            return False
        
        is_google_domain = any(hostname_lower.endswith(domain) for domain in GOOGLE_DOMAINS)
        
        if not is_google_domain:
            return False
        
        try:
            forward_ips = []
            
            try:
                forward_answers = resolver.resolve(hostname, 'A')
                forward_ips.extend([str(rdata) for rdata in forward_answers])
            except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
                pass
            
            try:
                forward_answers = resolver.resolve(hostname, 'AAAA')
                forward_ips.extend([str(rdata) for rdata in forward_answers])
            except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
                pass
            
            if not forward_ips:
                return False
            
            if ip in forward_ips:
                return True
            else:
                return False
        
        except (dns.resolver.Timeout, dns.exception.DNSException):
            return False
    
    except Exception:
        return False

def verify_ips_parallel_dns(ips_to_verify, desc="DNS Verification"):
    """Verify IPs using FcrDNS in parallel"""
    verified_ips = {}
    
    with ThreadPoolExecutor(max_workers=MAX_DNS_WORKERS) as executor:
        future_to_ip = {
            executor.submit(verify_ip_method2_dns_fast, ip): ip 
            for ip in ips_to_verify
        }
        
        with tqdm(total=len(ips_to_verify), desc=desc, unit=" IPs") as pbar:
            for future in as_completed(future_to_ip):
                ip = future_to_ip[future]
                try:
                    is_verified = future.result(timeout=DNS_TIMEOUT + 1)
                    verified_ips[ip] = is_verified
                except Exception:
                    verified_ips[ip] = False
                finally:
                    pbar.update(1)
    
    return verified_ips

# ============================================================================
# MAIN PROCESSING - ENHANCED 3-PHASE WITH FIXED LENIENT MODE
# ============================================================================

def process_log_files_enhanced(input_folder=None, input_file=None, output_folder="cleaned_logs_enhanced", 
                               max_records_per_file=500000, create_zip=True):
    """
    ENHANCED 3-PHASE GOOGLEBOT EXTRACTION (FIXED LENIENT MODE)
    
    Phase 1: Filter by Googlebot UA → ~18M records
    Phase 2: CIDR check + IMPROVED Lenient mode → ~15M+ records
    Phase 3: FcrDNS fallback for non-CIDR IPs → Final ~14.9M (99-100% match!)
    """
    
    print("=" * 80)
    print("🎯 ENHANCED 3-PHASE GOOGLEBOT EXTRACTION (FIXED)")
    print("=" * 80)
    print(f"   Lenient Mode: {'ENABLED' if LENIENT_MODE else 'DISABLED'}")
    print(f"   FcrDNS Fallback: {'ENABLED' if ENABLE_FCRDNS_FALLBACK else 'DISABLED'}")
    
    # Load IP ranges
    print(f"\n{'=' * 80}")
    print(f"LOADING GOOGLE IP RANGES")
    print(f"{'=' * 80}")
    
    try:
        google_ip_networks = load_or_fetch_ip_ranges()
        print(f"✅ Ready with {len(google_ip_networks)} official IP ranges")
    except Exception as e:
        print(f"❌ Failed to load IP ranges: {str(e)}")
        return None
    
    # Get input files
    log_files = []
    if input_file:
        if os.path.exists(input_file):
            log_files = [input_file]
        else:
            print(f"❌ File not found: {input_file}")
            return None
    elif input_folder:
        if os.path.exists(input_folder):
            log_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) 
                        if f.endswith('.log') or f.endswith('.log.gz')])
            if not log_files:
                print(f"❌ No .log files found in: {input_folder}")
                return None
        else:
            print(f"❌ Folder not found: {input_folder}")
            return None
    else:
        print("❌ Please provide input_folder or input_file")
        return None
    
    os.makedirs(output_folder, exist_ok=True)
    abs_output_folder = os.path.abspath(output_folder)
    
    # Create subfolders
    phase1_folder = os.path.join(output_folder, "phase1_ua_filtered")
    phase2_folder = os.path.join(output_folder, "phase2_cidr_filtered")
    phase3_folder = os.path.join(output_folder, "phase3_final_verified")
    os.makedirs(phase1_folder, exist_ok=True)
    os.makedirs(phase2_folder, exist_ok=True)
    os.makedirs(phase3_folder, exist_ok=True)
    
    print(f"\n📁 Input files: {len(log_files)}")
    
    print(f"\n🔍 ENHANCED 3-PHASE STRATEGY:")
    print(f"   Phase 1: Filter by Googlebot UA → ~18M")
    print(f"   Phase 2: CIDR + IMPROVED Lenient mode → ~15M+")
    print(f"   Phase 3: FcrDNS fallback → ~14.9M (TARGET!)")
    
    # ========================================================================
    # PHASE 1: FILTER BY GOOGLEBOT UA ONLY
    # ========================================================================
    print(f"\n{'=' * 80}")
    print(f"PHASE 1: FILTER BY GOOGLEBOT UA")
    print(f"{'=' * 80}")
    
    phase1_files = []
    total_phase1_records = 0
    unique_ips_phase1 = set()
    googlebot_variants = {}
    
    stats_phase1 = {
        'total_processed': 0,
        'has_googlebot_ua': 0,
    }
    
    start_phase1 = time.time()
    
    for file_idx, log_file in enumerate(log_files, 1):
        filename = os.path.basename(log_file)
        file_timestamp = extract_full_timestamp_from_filename(filename)
        
        if file_timestamp:
            base_filename = f"phase1_{file_timestamp}"
        else:
            base_filename = f"phase1_file{file_idx:04d}"
        
        file_size_mb = os.path.getsize(log_file) / (1024**2)
        print(f"\n📂 [{file_idx}/{len(log_files)}] {filename} ({file_size_mb:.1f} MB)")
        
        current_file_index = 1
        current_file_records = 0
        current_writer = None
        current_csvfile = None
        
        def create_new_csv_file(file_index):
            if file_index == 1:
                csv_filename = os.path.join(phase1_folder, f"{base_filename}.csv")
            else:
                csv_filename = os.path.join(phase1_folder, f"{base_filename}_part{file_index}.csv")
            
            csvfile = open(csv_filename, 'w', newline='', encoding='utf-8-sig')
            writer = csv.DictWriter(csvfile, fieldnames=[
                'timestamp', 'request_uri', 'http_x_forwarded_for', 
                'bytes_sent', 'upstream_response_time', 'user_agent'
            ], quoting=csv.QUOTE_ALL)
            writer.writeheader()
            phase1_files.append(csv_filename)
            return csvfile, writer
        
        current_csvfile, current_writer = create_new_csv_file(current_file_index)
        
        file_accepted = 0
        
        try:
            if log_file.endswith('.gz'):
                file_handle = gzip.open(log_file, 'rt', encoding='utf-8', errors='ignore')
            else:
                file_handle = open(log_file, 'r', encoding='utf-8', errors='ignore')
            
            with file_handle as log_reader:
                pbar = tqdm(log_reader, desc="   Filtering UA", unit=" recs", ncols=100, mininterval=0.5)
                
                for line in pbar:
                    try:
                        if not line.strip():
                            continue
                        
                        log_entry = json.loads(line.strip())
                        
                        stats_phase1['total_processed'] += 1
                        
                        user_agent = log_entry.get('http_user_agent', '')
                        if user_agent:
                            user_agent = str(user_agent).strip()
                        
                        # ONLY filter by UA in Phase 1
                        if not is_googlebot(user_agent):
                            continue
                        
                        stats_phase1['has_googlebot_ua'] += 1
                        
                        # Track variant
                        ua_lower = user_agent.lower()
                        for variant in ['googlebot-image', 'googlebot-news', 'googlebot-video', 
                                       'google-inspectiontool', 'adsbot-google', 'mediapartners-google']:
                            if variant in ua_lower:
                                key = variant.replace('-', ' ').title().replace(' ', '-')
                                googlebot_variants[key] = googlebot_variants.get(key, 0) + 1
                                break
                        else:
                            if 'googlebot' in ua_lower:
                                googlebot_variants['Googlebot (standard)'] = googlebot_variants.get('Googlebot (standard)', 0) + 1
                        
                        # Extract fields
                        request_uri = log_entry.get('request_uri', '')
                        if request_uri:
                            request_uri = str(request_uri).strip()
                        
                        timestamp = log_entry.get('time_iso8601', '')
                        if timestamp:
                            timestamp = str(timestamp).strip()
                        
                        ip = extract_first_ip(log_entry.get('http_x_forwarded_for', ''))
                        if ip:
                            unique_ips_phase1.add(ip)
                        
                        bytes_sent = log_entry.get('bytes_sent', '')
                        upstream_response_time = log_entry.get('upstream_response_time', '')
                        
                        # Partition check
                        if current_file_records >= max_records_per_file:
                            current_csvfile.close()
                            current_file_index += 1
                            current_csvfile, current_writer = create_new_csv_file(current_file_index)
                            current_file_records = 0
                        
                        csv_record = {
                            'timestamp': timestamp if timestamp else '',
                            'request_uri': request_uri if request_uri else '',
                            'http_x_forwarded_for': ip if ip else '',
                            'bytes_sent': bytes_sent if bytes_sent else '',
                            'upstream_response_time': upstream_response_time if upstream_response_time else '',
                            'user_agent': user_agent if user_agent else ''
                        }
                        
                        current_writer.writerow(csv_record)
                        current_file_records += 1
                        file_accepted += 1
                        total_phase1_records += 1
                    
                    except Exception:
                        continue
                
                pbar.close()
            
            if current_csvfile and not current_csvfile.closed:
                current_csvfile.close()
            
            print(f"   ✅ Accepted: {file_accepted:,} records with Googlebot UA")
        
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            if current_csvfile and not current_csvfile.closed:
                current_csvfile.close()
            continue
    
    elapsed_phase1 = time.time() - start_phase1
    
    print(f"\n{'=' * 80}")
    print(f"✅ PHASE 1 COMPLETE")
    print(f"{'=' * 80}")
    print(f"   Total processed: {stats_phase1['total_processed']:,}")
    print(f"   Googlebot UA filtered: {total_phase1_records:,}")
    print(f"   Unique IPs: {len(unique_ips_phase1):,}")
    print(f"   Time: {elapsed_phase1:.1f}s ({elapsed_phase1/60:.1f} min)")
    
    # ========================================================================
    # PHASE 2: CHECK IPS IN CIDR RANGES (IMPROVED LENIENT MODE)
    # ========================================================================
    print(f"\n{'=' * 80}")
    print(f"PHASE 2: CHECK IPS AGAINST CIDR RANGES (IMPROVED LENIENT MODE)")
    print(f"{'=' * 80}")
    
    print(f"\n🔍 Checking {len(unique_ips_phase1):,} IPs against CIDR ranges...")
    verified_ips_cidr = {}
    for ip in tqdm(unique_ips_phase1, desc="Checking IPs", unit=" IPs"):
        verified_ips_cidr[ip] = verify_ip_in_range(ip, google_ip_networks)
    
    ips_in_range = sum(1 for v in verified_ips_cidr.values() if v)
    ips_outside_range = len(unique_ips_phase1) - ips_in_range
    print(f"   ✅ IPs in CIDR ranges: {ips_in_range:,}/{len(unique_ips_phase1):,}")
    print(f"   ⚠️ IPs outside ranges: {ips_outside_range:,} (will check with FcrDNS in Phase 3)")
    
    print(f"\n📝 Writing CIDR-filtered records (IMPROVED LENIENT MODE)...")
    
    phase2_files = []
    total_phase2_records = 0
    unique_ips_phase2 = set()
    unique_ips_outside_cidr = set()
    records_missing_ip = 0
    records_invalid_ip = 0  # NEW: Track invalid IPs
    
    start_phase2 = time.time()
    
    for phase1_file in tqdm(phase1_files, desc="Processing Phase 1 files", unit=" files"):
        base_name = os.path.basename(phase1_file).replace('phase1_', 'phase2_')
        
        current_file_index = 1
        current_file_records = 0
        current_writer = None
        current_csvfile = None
        
        def create_phase2_file(file_index):
            if file_index == 1:
                csv_filename = os.path.join(phase2_folder, base_name)
            else:
                name_parts = base_name.rsplit('.', 1)
                csv_filename = os.path.join(phase2_folder, f"{name_parts[0]}_part{file_index}.{name_parts[1]}")
            
            csvfile = open(csv_filename, 'w', newline='', encoding='utf-8-sig')
            writer = csv.DictWriter(csvfile, fieldnames=[
                'timestamp', 'request_uri', 'http_x_forwarded_for', 
                'bytes_sent', 'upstream_response_time', 'ip_status'
            ], quoting=csv.QUOTE_ALL)
            writer.writeheader()
            phase2_files.append(csv_filename)
            return csvfile, writer
        
        current_csvfile, current_writer = create_phase2_file(current_file_index)
        
        try:
            with open(phase1_file, 'r', encoding='utf-8-sig') as infile:
                reader = csv.DictReader(infile)
                
                for row in reader:
                    ip = row.get('http_x_forwarded_for', '').strip()  # FIXED: Added .strip()
                    
                    # Determine IP status
                    ip_status = 'unknown'
                    accept_record = False
                    
                    # IMPROVED: Check for various "missing" states
                    if not ip or ip == '' or ip.lower() in ['none', 'null', '-', 'unknown']:
                        # Missing or invalid IP but has Googlebot UA (lenient)
                        if LENIENT_MODE:
                            accept_record = True
                            ip_status = 'missing_ip_lenient'
                            records_missing_ip += 1
                    
                    # Check if IP is in our verified list
                    elif ip in verified_ips_cidr:
                        if verified_ips_cidr[ip]:
                            # IP in CIDR range
                            accept_record = True
                            ip_status = 'in_cidr'
                            unique_ips_phase2.add(ip)
                        else:
                            # IP NOT in CIDR - will verify with FcrDNS later
                            accept_record = True
                            ip_status = 'needs_fcrdns'
                            unique_ips_outside_cidr.add(ip)
                            unique_ips_phase2.add(ip)
                    
                    else:
                        # IP not in Phase 1 unique list - might be malformed
                        if LENIENT_MODE:
                            accept_record = True
                            ip_status = 'invalid_ip_lenient'
                            records_invalid_ip += 1
                    
                    if accept_record:
                        if current_file_records >= max_records_per_file:
                            current_csvfile.close()
                            current_file_index += 1
                            current_csvfile, current_writer = create_phase2_file(current_file_index)
                            current_file_records = 0
                        
                        current_writer.writerow({
                            'timestamp': row.get('timestamp', ''),
                            'request_uri': row.get('request_uri', ''),
                            'http_x_forwarded_for': ip if ip else '',
                            'bytes_sent': row.get('bytes_sent', ''),
                            'upstream_response_time': row.get('upstream_response_time', ''),
                            'ip_status': ip_status
                        })
                        current_file_records += 1
                        total_phase2_records += 1
            
            if current_csvfile and not current_csvfile.closed:
                current_csvfile.close()
        
        except Exception:
            if current_csvfile and not current_csvfile.closed:
                current_csvfile.close()
            continue
    
    elapsed_phase2 = time.time() - start_phase2
    
    print(f"\n{'=' * 80}")
    print(f"✅ PHASE 2 COMPLETE")
    print(f"{'=' * 80}")
    print(f"   Total records: {total_phase2_records:,}")
    print(f"   - In CIDR ranges: {ips_in_range:,} unique IPs")
    print(f"   - Missing IP (lenient): {records_missing_ip:,}")
    print(f"   - Invalid IP (lenient): {records_invalid_ip:,}")
    print(f"   - Outside CIDR (needs FcrDNS): {len(unique_ips_outside_cidr):,} unique IPs")
    print(f"   Phase 2 files: {len(phase2_files)}")
    print(f"   Time: {elapsed_phase2:.1f}s ({elapsed_phase2/60:.1f} min)")
    
    # ========================================================================
    # PHASE 3: FcrDNS VERIFICATION FOR NON-CIDR IPs
    # ========================================================================
    print(f"\n{'=' * 80}")
    print(f"PHASE 3: FcrDNS VERIFICATION (FOR NON-CIDR IPs)")
    print(f"{'=' * 80}")
    
    dns_verified_ips = {}
    
    if ENABLE_FCRDNS_FALLBACK and len(unique_ips_outside_cidr) > 0:
        print(f"\n🌐 Starting FcrDNS verification for {len(unique_ips_outside_cidr):,} IPs outside CIDR ranges...")
        print(f"   Using {MAX_DNS_WORKERS} workers")
        
        start_phase3_dns = time.time()
        
        dns_verified_ips = verify_ips_parallel_dns(unique_ips_outside_cidr, desc="FcrDNS Verification")
        
        dns_pass = sum(1 for v in dns_verified_ips.values() if v)
        elapsed_dns = time.time() - start_phase3_dns
        
        print(f"\n✅ FcrDNS Verification Results:")
        print(f"   Passed: {dns_pass:,}/{len(unique_ips_outside_cidr):,}")
        print(f"   Time: {elapsed_dns:.1f}s ({elapsed_dns/60:.1f} min)")
        if len(unique_ips_outside_cidr) > 0:
            print(f"   Speed: {len(unique_ips_outside_cidr)/elapsed_dns:.0f} IPs/sec")
    else:
        print(f"\n⚠️ FcrDNS verification skipped")
    
    print(f"\n📝 Writing final verified records...")
    
    phase3_files = []
    total_phase3_records = 0
    rejection_stats = {
        'in_cidr': 0,
        'missing_ip_accepted': 0,
        'invalid_ip_accepted': 0,  # NEW
        'fcrdns_passed': 0,
        'fcrdns_failed_rejected': 0
    }
    
    start_phase3_write = time.time()
    
    for phase2_file in tqdm(phase2_files, desc="Processing Phase 2 files", unit=" files"):
        base_name = os.path.basename(phase2_file).replace('phase2_', 'phase3_')
        phase3_file = os.path.join(phase3_folder, base_name)
        
        try:
            with open(phase2_file, 'r', encoding='utf-8-sig') as infile:
                reader = csv.DictReader(infile)
                
                with open(phase3_file, 'w', newline='', encoding='utf-8-sig') as outfile:
                    writer = csv.DictWriter(outfile, fieldnames=[
                        'timestamp', 'request_uri', 'http_x_forwarded_for', 
                        'bytes_sent', 'upstream_response_time'
                    ], quoting=csv.QUOTE_ALL)
                    writer.writeheader()
                    
                    for row in reader:
                        ip = row.get('http_x_forwarded_for', '').strip()  # FIXED: Added .strip()
                        ip_status = row.get('ip_status', '')
                        
                        accept_record = False
                        
                        if ip_status == 'in_cidr':
                            accept_record = True
                            rejection_stats['in_cidr'] += 1
                        
                        elif ip_status == 'missing_ip_lenient':
                            accept_record = True
                            rejection_stats['missing_ip_accepted'] += 1
                        
                        elif ip_status == 'invalid_ip_lenient':  # NEW: Accept invalid/malformed IPs
                            accept_record = True
                            rejection_stats['invalid_ip_accepted'] += 1
                        
                        elif ip_status == 'needs_fcrdns':
                            if dns_verified_ips.get(ip, False):
                                accept_record = True
                                rejection_stats['fcrdns_passed'] += 1
                            else:
                                accept_record = False
                                rejection_stats['fcrdns_failed_rejected'] += 1
                        
                        if accept_record:
                            writer.writerow({
                                'timestamp': row.get('timestamp', ''),
                                'request_uri': row.get('request_uri', ''),
                                'http_x_forwarded_for': ip,
                                'bytes_sent': row.get('bytes_sent', ''),
                                'upstream_response_time': row.get('upstream_response_time', '')
                            })
                            total_phase3_records += 1
            
            phase3_files.append(phase3_file)
        except Exception:
            continue
    
    elapsed_phase3_write = time.time() - start_phase3_write
    
    print(f"\n{'=' * 80}")
    print(f"✅ PHASE 3 COMPLETE")
    print(f"{'=' * 80}")
    print(f"   Final records: {total_phase3_records:,}")
    print(f"   Breakdown:")
    print(f"   - CIDR verified: {rejection_stats['in_cidr']:,}")
    print(f"   - Missing IP (lenient): {rejection_stats['missing_ip_accepted']:,}")
    print(f"   - Invalid IP (lenient): {rejection_stats['invalid_ip_accepted']:,}")
    print(f"   - FcrDNS passed: {rejection_stats['fcrdns_passed']:,}")
    print(f"   - FcrDNS failed (rejected): {rejection_stats['fcrdns_failed_rejected']:,}")
    print(f"   Output files: {len(phase3_files)}")
    
    # ========================================================================
    # SUMMARY
    # ========================================================================
    total_time = elapsed_phase1 + elapsed_phase2
    if ENABLE_FCRDNS_FALLBACK and len(unique_ips_outside_cidr) > 0:
        total_time += elapsed_dns + elapsed_phase3_write
    
    gsc_expected = 14913024
    match_rate = (total_phase3_records / gsc_expected * 100) if total_phase3_records > 0 else 0
    
    print(f"\n{'=' * 80}")
    print(f"✅ COMPLETE: ENHANCED 3-PHASE EXTRACTION (FIXED)")
    print(f"{'=' * 80}")
    
    print(f"\n⏱️  PERFORMANCE:")
    print(f"   Phase 1 (UA Filter): {elapsed_phase1:.1f}s ({elapsed_phase1/60:.1f} min)")
    print(f"   Phase 2 (CIDR + Lenient): {elapsed_phase2:.1f}s ({elapsed_phase2/60:.1f} min)")
    if ENABLE_FCRDNS_FALLBACK and len(unique_ips_outside_cidr) > 0:
        print(f"   Phase 3 (FcrDNS): {(elapsed_dns + elapsed_phase3_write):.1f}s ({(elapsed_dns + elapsed_phase3_write)/60:.1f} min)")
    print(f"   TOTAL: {total_time:.1f}s ({total_time/60:.1f} min)")
    
    print(f"\n📊 RESULTS:")
    print(f"   Phase 1: {total_phase1_records:,} (Googlebot UA)")
    print(f"   Phase 2: {total_phase2_records:,} (CIDR + Lenient)")
    print(f"   Phase 3: {total_phase3_records:,} (Final Verified)")
    
    print(f"\n🤖 GOOGLEBOT VARIANTS:")
    for variant, count in sorted(googlebot_variants.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"   • {variant}: {count:,}")
    
    print(f"\n🎯 GSC COMPARISON:")
    print(f"   GSC: {gsc_expected:,}")
    print(f"   Output: {total_phase3_records:,}")
    print(f"   Difference: {abs(gsc_expected - total_phase3_records):,}")
    print(f"   Match: {match_rate:.1f}%")
    
    if match_rate >= 98:
        print(f"   🏆 NEAR-PERFECT!")
    elif match_rate >= 95:
        print(f"   ✅ EXCELLENT MATCH!")
    elif match_rate >= 90:
        print(f"   ✅ VERY GOOD!")
    
    # Create ZIP
    if create_zip:
        zip_filename = create_zip_archive(phase3_files, output_folder, "googlebot_final")
    
    return {
        'output_folder': abs_output_folder,
        'phase3_files': phase3_files,
        'final_records': total_phase3_records,
        'match_rate': match_rate,
        'rejection_stats': rejection_stats
    }

# ============================================================================
# USAGE
# ============================================================================

print("=" * 80)
print("🎯 ENHANCED 3-PHASE GOOGLEBOT EXTRACTION (FIXED)")
print("   Phase 1: UA | Phase 2: CIDR + IMPROVED Lenient | Phase 3: FcrDNS")
print("=" * 80)

INPUT_FOLDER = input("\n📂 Enter folder path (or empty for file): ").strip()

if not INPUT_FOLDER:
    INPUT_FILE = input("📄 Enter file path: ").strip()
    results = process_log_files_enhanced(
        input_file=INPUT_FILE,
        output_folder="cleaned_logs_enhanced_fixed",
        max_records_per_file=500000,
        create_zip=True
    )
else:
    results = process_log_files_enhanced(
        input_folder=INPUT_FOLDER,
        output_folder="cleaned_logs_enhanced_fixed",
        max_records_per_file=500000,
        create_zip=True
    )

if results:
    print(f"\n🎉 DONE!")
    print(f"📂 {results['output_folder']}")
    print(f"📋 Final: {results['final_records']:,} records")
    print(f"🎯 Match: {results['match_rate']:.1f}%")


🎯 ENHANCED 3-PHASE GOOGLEBOT EXTRACTION (FIXED)
   Phase 1: UA | Phase 2: CIDR + IMPROVED Lenient | Phase 3: FcrDNS



📂 Enter folder path (or empty for file):  D:\Task-3 complete\dayfiles_9thoct2025


🎯 ENHANCED 3-PHASE GOOGLEBOT EXTRACTION (FIXED)
   Lenient Mode: ENABLED
   FcrDNS Fallback: ENABLED

LOADING GOOGLE IP RANGES

✅ Using cached IP ranges (age: 18.7 hours)
   • IPv4 ranges: 982
   • IPv6 ranges: 959
✅ Ready with 1941 official IP ranges

📁 Input files: 24

🔍 ENHANCED 3-PHASE STRATEGY:
   Phase 1: Filter by Googlebot UA → ~18M
   Phase 2: CIDR + IMPROVED Lenient mode → ~15M+
   Phase 3: FcrDNS fallback → ~14.9M (TARGET!)

PHASE 1: FILTER BY GOOGLEBOT UA

📂 [1/24] nginx-20251008_000000.log (4538.5 MB)


   Filtering UA: 2551183 recs [03:22, 12605.47 recs/s]


   ✅ Accepted: 773,128 records with Googlebot UA

📂 [2/24] nginx-20251008_010000.log (4529.1 MB)


...


✅ FcrDNS Verification Results:
   Passed: 0/394
   Time: 6.3s (0.1 min)
   Speed: 62 IPs/sec

📝 Writing final verified records...


Processing Phase 2 files: 100%|████████████████████████████████████████████████████| 49/49 [03:41<00:00,  4.53s/ files]



✅ PHASE 3 COMPLETE
   Final records: 14,259,928
   Breakdown:
   - CIDR verified: 14,259,928
   - Missing IP (lenient): 0
   - Invalid IP (lenient): 0
   - FcrDNS passed: 0
   - FcrDNS failed (rejected): 3,669,684
   Output files: 49

✅ COMPLETE: ENHANCED 3-PHASE EXTRACTION (FIXED)

⏱️  PERFORMANCE:
   Phase 1 (UA Filter): 2685.5s (44.8 min)
   Phase 2 (CIDR + Lenient): 369.8s (6.2 min)
   Phase 3 (FcrDNS): 228.1s (3.8 min)
   TOTAL: 3283.5s (54.7 min)

📊 RESULTS:
   Phase 1: 17,929,612 (Googlebot UA)
   Phase 2: 17,929,612 (CIDR + Lenient)
   Phase 3: 14,259,928 (Final Verified)

🤖 GOOGLEBOT VARIANTS:
   • Googlebot (standard): 16,523,241
   • Adsbot-Google: 1,240,612
   • Googlebot-Image: 2,789
   • Mediapartners-Google: 1,152

🎯 GSC COMPARISON:
   GSC: 14,913,024
   Output: 14,259,928
   Difference: 653,096
   Match: 95.6%
   ✅ EXCELLENT MATCH!

📦 Creating ZIP archive...


Archiving: 100%|███████████████████████████████████████████████████████████████████| 49/49 [02:12<00:00,  2.71s/ files]

✅ ZIP created: googlebot_final_20251113_100009.zip (964.91 MB)

🎉 DONE!
📂 C:\Users\MadhavN\cleaned_logs_enhanced_fixed
📋 Final: 14,259,928 records
🎯 Match: 95.6%
